In [2]:
from langchain_groq import ChatGroq

In [1]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name='llama-3.1-70b-versatile'
)
# invoke to use llm
output_response = llm.invoke("The first Indian woman Officer")
print(output_response.content)

/Users/vishal/Documents/GenAI/genai/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


The first Indian woman officer in the Indian Armed Forces was Captain Lakshmi Sahgal (also known as Lakshmi Swaminathan). However, she was part of the Indian National Army (INA) during World War II.

The first Indian woman officer in the regular Indian Armed Forces was Second Lieutenant Kiran Shekhawat, who was commissioned into the Indian Army in 1992.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-34397?from=job%20search%20funnel")
page_data = loader.load().pop().page_content

In [8]:
print(page_data)

Apply for LEAD, GLOBAL PROCUREMENT OUTBOUND MIDDLE MILE CATEGORY MANAGER

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsProcurementLEAD, GLOBAL PROCUREMENT OUTBOUND MIDDLE MILE CATEGOR

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    '''
    ### SCRAPED TEXT FROM WEBSITE: 
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings nd return them in JSON format containing following keys : 
    'role','skills' and 'description'.
    Only return the valid JSON. 
    ### VALID JSON (NO PREAMBLE):
    '''
)

# prompt passing to llm
chain_prompt = prompt_extract | llm
response = chain_prompt.invoke(input={'page_data':page_data})

In [11]:
print(response.content)

```
{
  "role": "LEAD, GLOBAL PROCUREMENT OUTBOUND MIDDLE MILE CATEGORY MANAGER",
  "skills": [
    "Bachelor’s degree or equivalent combination of education, experience or training",
    "6+ years work experience in Procurement and/or Transportation, with a focus on Category management and/or Strategic Sourcing with expertise in transportation for consumer goods or retail",
    "Deep understanding of the transportation market and modes of transportation. Key categories include: Full Truckload, Less-Than-Truckload, Pool Point, Drayage, Intermodal and Small Parcel",
    "Deep understanding of P&L for a logistics service provider",
    "Experience with clean sheet should cost modeling and negotiation methodology",
    "Total Cost of Ownership approach to spend management",
    "Rigorous cost savings approaches and analytics, with expertise in total delivered cost",
    "Familiarity with product management and development approaches, including consumer insights and competitive product ana

In [14]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)

In [15]:
json_response

{'role': 'LEAD, GLOBAL PROCUREMENT OUTBOUND MIDDLE MILE CATEGORY MANAGER',
 'skills': ['Bachelor’s degree or equivalent combination of education, experience or training',
  '6+ years work experience in Procurement and/or Transportation, with a focus on Category management and/or Strategic Sourcing with expertise in transportation for consumer goods or retail',
  'Deep understanding of the transportation market and modes of transportation. Key categories include: Full Truckload, Less-Than-Truckload, Pool Point, Drayage, Intermodal and Small Parcel',
  'Deep understanding of P&L for a logistics service provider',
  'Experience with clean sheet should cost modeling and negotiation methodology',
  'Total Cost of Ownership approach to spend management',
  'Rigorous cost savings approaches and analytics, with expertise in total delivered cost',
  'Familiarity with product management and development approaches, including consumer insights and competitive product analysis',
  'Experience worki

In [17]:
import pandas as pd 

df = pd.read_csv("tech_portfolios.csv")
df.head()


,TechStack,Portfolio_links
0,Backend,https://behance.net/user1
1,Go,https://behance.net/user2
2,DevOps,https://medium.com/user3
3,Laravel,https://dribbble.com/user4
4,HTML/CSS,https://medium.com/user5


In [18]:
import chromadb

# Using PersistentClient to create db on disc , Client creates it in memory.
client = chromadb.PersistentClient('vector_store')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count:
   for _, row in df.iterrows():
      collection.add(
         documents = row['TechStack'],
         metadatas = {"links":row['Portfolio_links']},
         ids = [str(uuid.uuid4())]
      )

In [27]:
links = collection.query(query_texts=['Backend', 'HTML/CSS'],n_results=2).get('metadatas',[])
links

[[], []]

In [22]:
job = json_response
json_response['skills']

['Bachelor’s degree or equivalent combination of education, experience or training',
 '6+ years work experience in Procurement and/or Transportation, with a focus on Category management and/or Strategic Sourcing with expertise in transportation for consumer goods or retail',
 'Deep understanding of the transportation market and modes of transportation. Key categories include: Full Truckload, Less-Than-Truckload, Pool Point, Drayage, Intermodal and Small Parcel',
 'Deep understanding of P&L for a logistics service provider',
 'Experience with clean sheet should cost modeling and negotiation methodology',
 'Total Cost of Ownership approach to spend management',
 'Rigorous cost savings approaches and analytics, with expertise in total delivered cost',
 'Familiarity with product management and development approaches, including consumer insights and competitive product analysis',
 'Experience working cross-culturally on global categories, with leadership, influencing and communication skill

In [29]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    '''
    ### SCRAPED TEXT FROM WEBSITE: 
    {job_description}
    ### INSTRUCTION:
    You are Keshav, a business development executive at Microsoft. Microsoft is an service based company that provides assets in form of services
    and products be it Hardware or Software. Over our experience we have empowered numerous enterprises ith tailored solutions.Your job is to write a cold 
    email to the client regarding he job mentioned above in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Microsoft's portfolio : {link_list}
    Remember You are Keshav , BDE at Microsoft.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    '''
)

chain_email = prompt_extract | llm
response_cold_email = chain_email.invoke(input={'job_description':str(job),'link_list':links})

In [30]:
print(response_cold_email.content)

Subject: Unlock Efficient Global Procurement with Microsoft Solutions

Dear [Hiring Manager's Name],

I came across the job posting for the Lead, Global Procurement Outbound Middle Mile Category Manager role and was impressed by the organization's commitment to strategic sourcing and category management. As a Business Development Executive at Microsoft, I'd like to introduce you to our suite of solutions that can help your team streamline procurement processes, drive cost savings, and enhance collaboration.

Our Azure-based procurement solutions, such as Azure Supply Chain Platform and Dynamics 365 Supply Chain Management, can help you:

* Optimize transportation modes and routes with advanced analytics and AI-powered insights
* Automate procurement workflows and improve supplier management with Azure-based e-Sourcing tools
* Enhance collaboration and visibility across global teams with Microsoft Teams and Power BI
* Develop a Total Cost of Ownership approach to spend management with o